In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import datetime, os
import matplotlib.pyplot as plt
from math import floor
from numpy import expand_dims
from numpy import log
from numpy import mean
from numpy import std
from numpy import exp

In [ ]:
# we first upload zip files (VAE_encoder.zip, VAE_decoder.zip) to colab 

In [ ]:
# unzip saved models to load into keras object
!unzip /content/VAE_encoder.zip
!unzip /content/VAE_decoder.zip

In [ ]:
# load encoder and decoder models into keras objects
encoder = keras.models.load_model("/content/VAE_encoder") 
decoder = keras.models.load_model("/content/VAE_decoder") 

In [ ]:
def generate_images(decoder):
  # generate images from random vectors
  latent_dim = 50
  num = 100 #number of images to generate
  num_row = 10
  num_col = 10
  digit_size = 28

  # draw std. normal variables from latent space
  np.random.seed(0)
  rand_vars = np.random.normal(size=(num,latent_dim))

  # plot images
  fig, axes = plt.subplots(num_row, num_col, figsize=(1.5*num_col,2*num_row))
  for i in range(num):
      z_sample = np.array([rand_vars[i]])
      x_decoded = decoder.predict(z_sample)
      digit = x_decoded[0].reshape(digit_size, digit_size)

      ax = axes[i//num_col, i%num_col]
      ax.imshow(digit, cmap='gray')
  plt.tight_layout()
  plt.show()

In [ ]:
generate_images(decoder)

In [ ]:
# find inception score to evaluate generated images 

In [ ]:
def get_images():
  latent_dim = 50
  num = 100 #number of images to generate
  num_row = 10
  num_col = 10
  digit_size = 28

  # draw std. normal variables from latent space
  np.random.seed(0)
  rand_vars = np.random.normal(size=(num,latent_dim))
  x_decoded = decoder.predict(rand_vars)
  return x_decoded

In [ ]:
def generate_mnist_classifier():

  # Model / data parameters
  num_classes = 10
  input_shape = (28, 28, 1)

  # the data, split between train and test sets
  (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

  # Scale images to the [0, 1] range
  x_train = x_train.astype("float32") / 255
  x_test = x_test.astype("float32") / 255
  # Make sure images have shape (28, 28, 1)
  x_train = np.expand_dims(x_train, -1)
  x_test = np.expand_dims(x_test, -1)
  print("x_train shape:", x_train.shape)
  print(x_train.shape[0], "train samples")
  print(x_test.shape[0], "test samples")


  # convert class vectors to binary class matrices
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)

  model = keras.Sequential(
      [
          keras.Input(shape=input_shape),
          layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
          layers.MaxPooling2D(pool_size=(2, 2)),
          layers.Flatten(),
          layers.Dropout(0.5),
          layers.Dense(num_classes, activation="softmax"),
      ]
  )

  model.summary()

  batch_size = 128
  epochs = 10

  model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
  model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
  return model

In [ ]:
def calculate_inception_score(images, model, n_split=10, eps=1E-16):

	# enumerate splits of images/predictions
	scores = list()
	n_part = floor(images.shape[0] / n_split)
	for i in range(n_split):
		# retrieve images
		ix_start, ix_end = i * n_part, (i+1) * n_part
		subset = images[ix_start:ix_end]
		# predict p(y|x)
		p_yx = model.predict(subset)
		# calculate p(y)
		p_y = expand_dims(p_yx.mean(axis=0), 0)
		# calculate KL divergence using log probabilities
		kl_d = p_yx * (log(p_yx + eps) - log(p_y + eps))
		# sum over classes
		sum_kl_d = kl_d.sum(axis=1)
		# average over images
		avg_kl_d = mean(sum_kl_d)
		# undo the log
		is_score = exp(avg_kl_d)
		# store
		scores.append(is_score)
	# average across images
	is_avg, is_std = mean(scores), std(scores)
	return is_avg, is_std

In [ ]:
images = get_images()

In [ ]:
model = generate_mnist_classifier()

In [ ]:
calculate_inception_score(images, model)